## Homework 08:  Parallel Programming 01

## Due Date: Apr 12, 2023, 11:59pm

#### Firstname Lastname: Buz Galbraith

#### E-mail: wbg231@nyu.edu

#### Enter your solutions and submit this notebook

---

**Problem 1 (50p)**

Write an MPI program `sol08pr01.py` that does the following for some arbitrary number of processes $N \geq 2$. Here the number of processes $N$ is given as `N` while calling the code `sol08pr01.py` as: 

`mpirun -n N python3 sol08pr01.py`


Every process will contain one buffer with one integer variable, each of which is initialized to $0$.

For $r=0, 1, \dots, N - 1$, Process $r$ updates its buffer to the value received by $r-1$ (this should only be done for $r \geq 1$), then it squares its rank $r$, adds the result $r^2$ to the value of its own buffer, and then sends the sum to Process $r + 1$. Note that for $r=N-1$ the result will be sent to Process $0$, i.e. by convention, Process $N$ is the same as Process $0$. At the end Process $0$ prints the received value. 

Provide results for: $N = 10$, $N = 15$, $N = 20$, $N = 25$. These are probably more than the available processes on your machine: you can use the option `--oversubscribe` in `mpirun` to let MPI run things anyway.



**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code. 



In [39]:
%%writefile sol08pr01.py

from mpi4py import MPI
import numpy as np

## i think one needs to use non-blocking but with a lot of wait statements so it is equivalent to explicit blocking, which is needed 
## since all values depend on the previous

def worker_node(rank,size):
    src=rank-1  ## where input comes from
    req = comm.Irecv(buf, source=src) ## receive input
    req.Wait() ## blocking wait statement (so same as com.wait())
    buf[0] += rank**2 ## adds square of it's own rank to the buffer
    dest = (rank + 1)%size ## calculates where to send it's output, use modulus size so that final rank sends to process 0.
    comm.Isend(buf, dest=dest) ##sends value without explicit blocking. 
def terminal_node(rank, size):
    dest=  (rank + 1)%size ## calculates where to send it's output, use modulus size so that final rank sends to process 0.
    comm.Isend(buf, dest=dest) ## sends initial buffer to process one 
    src=size-1 ## sets source to final rank
    req = comm.Irecv(buf, source=src) ## takes final value 
    req.Wait() ## blocking statement 
    print("Terminal Value  : {0}".format(buf[0])) ## prints output
 


comm = MPI.COMM_WORLD ## initialize communicator
size = comm.Get_size()
rank = comm.Get_rank() 
buf = np.zeros(1)  # buffer with a single integer variable initialized to zero
if rank > 0: ## worker node condition 
    worker_node(rank,size)
if rank == 0: ## terminal node condition 
    terminal_node(rank, size)

Overwriting sol08pr01.py


## n=10
Terminal Value  : 285.0

## n=15
Terminal Value  : 1015.0

## n=20
Terminal Value  : 2470.0

## n=25
Terminal Value  : 2470.0

---

**Problem 2 (50p)**

Write an MPI program that does the following. There are two processes 0 and 1 that have to exchange $T=10$ messages.  


Process 0 initially reads two float variables from the standard input, call them $x, y$, and must ensure $x \neq 0$ and $y \neq 0$. For example this can be done as:

```
import sys


for line in sys.stdin:
    x = float(line)        
    if x != 0.0:
        break
for line in sys.stdin:
    y = float(line)        
    if y != 0.0:
        break
```


Both Process 0 and Process 1 will carry main results in an element that is part of a process buffer and called $p$. The value in $p$ is initially set to $1$. 


Now the exchange of messages is as follows.


0. Message00: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

1. Message01: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

2. Message01: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

3. Message02: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.


etc.

8. Message08: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

9. Message09: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

Finally, Process 0 prints the value in $p$ as a final result. 


Write the code that implements the protocol above. Additionally, provide results for: $(x, y) = (2, 4)$, $(x, y) = (1, 3)$, $(x, y) = (5, 7)$ and $(x, y) = (5, 10)$.


**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code.



In [105]:
%%writefile sol08pr02.py
from mpi4py import MPI
import numpy as np
import sys




comm = MPI.COMM_WORLD
def parse_xy():
    for line in sys.stdin: ## read line in from standard input 
        try: ## try to convert to float
            x=float(line)
        except: ## if this does not work return an error message and abort program 
            print("x should be real number")  
            comm.Abort()      
        if(x==0): ## if x is 0 return an error message and abort program 
            print("x can not equal 0")
            comm.Abort()
        else:
            break
    for line in sys.stdin: ## does the same for y 
        try:
            y=float(line)
        except:
            print("y should be real number")  
            comm.Abort()      
        if(y==0):
            print("y can not equal 0")
            comm.Abort()
        else:
            break
    return x,y
def process_0(X):
        X[2]=X[2]*X[0] ## change the value of p 
        comm.Send(X, dest=1) ## send to rank 1, with blocking
        comm.Recv(X, source=1) ## receive from rank 1 with blocking 
        return X ## return buffer
def process_1(X):
    comm.Recv(X, source=0) ## receive from rank 0 with blocking
    X[2]=X[2]/X[1] ## update p value
    comm.Send(X, dest=0) ## send to rank 1 
    if(i==4):  ## if terminal case end
        print("terminal value : {0}".format(X[2])) ## print value
    return X ## return 

rank = comm.Get_rank()
X=np.zeros(3) ## initialize buffer of form [x,y,p]
X[2]=1 ## set p to initial value of 1. 
for i in range(5): # loop 5 times 
    if(rank==0): ## rank 0 case 
        if(i==0): ## is the first process so needs to initialize x,y
            X[0],X[1]=parse_xy() ## function to pase x,y and do some exception handling then set them 2 the x[0], x[1] val
        X=process_0(X) ## process 1 function set to buffer
    else: ## rank 1 case
        X=process_1(X) ## process 2 function set to buffer
    

Overwriting sol08pr02.py
